In [ ]:
!pip install nltk
!pip install --upgrade spacy
!pip install --upgrade spacy[cuda111,transformers]

!pip install newspaper3k
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from tqdm.autonotebook import tqdm
from spacy import displacy
from newspaper import Article
import re
import spacy
import nltk
import random
import string
import numpy as np
nlp = spacy.load("en_core_web_lg")
nltk.download('punkt', quiet=True)

# scraping and downloading the data from the website

article=Article('https://www.gov.uk/student-visa')
article.download()
article.parse()
article.nlp()

corpus = article.text

# Text Pre-processing

def clean_text(corpus):
  newline_re = r'[\n]+'
  white_space_re = r"\s+"

  corpus = re.sub(newline_re, r' ', corpus, flags=re.UNICODE)
  corpus = re.sub(white_space_re, r' ', corpus, flags=re.UNICODE)

  return corpus

def all_steps(text, steps):
  steps_dict = {'clean': clean_text}

  if len(steps):
    for step in steps:
      text = steps_dict[step](text)
  return text

text = corpus
steps = ['clean']
data = all_steps(text, steps)

text = data

sentence_list = nltk.sent_tokenize(text)

doc = nlp(text)

#passing the data in the pipeline

just_text = nltk.word_tokenize(text)
docs = list(tqdm(nlp.pipe(just_text), total=len(just_text)))

[(i.text, i.ent_iob_ + "-" + i.ent_type_) for i in doc[0:30]]

doc = nlp(text)
sent = list(doc.sents)[1]

nlp_trf = spacy.load("en_core_web_trf")

doc = nlp_trf(text)

# Chatbot functionality implementation

model_name = "deepset/roberta-base-squad2"

hugg = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [ ]:
# Question and Answering

QA_input = {
    'question': "what is skilled worker visa?",
    'context': text
}
res = hugg(QA_input)

print(res)